In [ ]:
import pytrec_eval
import numpy as np
from typing import Dict
import pandas as pd
from pyserini.search import LuceneSearcher
import seaborn as sns
queries_file = "data/proc_data/train_sample/sample_queries.tsv"
qrels_file = "data/proc_data/train_sample/sample_qrels.tsv"

In [ ]:
topics = pd.read_csv('data/proc_data/train_sample/sample_queries.tsv', sep=' ', names=['qid', 'query'])
qrels = pd.read_csv('data/proc_data/train_sample/sample_qrels.tsv', sep=' ', names=['qid', 'Q0', 'docid', 'rel'])

In [ ]:
print(topics.shape)
topics.sample(5)

In [ ]:
qrels.drop('Q0', axis=1, inplace=True)
qrels.drop('rel', axis=1, inplace=True)
print(qrels.shape)
qrels.sample(5)


In [ ]:
def create_dummy_qrels() -> Dict[str, Dict[str, int]]:
    return {
        "1": {"doc1": 1, "doc2": 0, "doc3": 1},
        "2": {"doc1": 0, "doc2": 1, "doc3": 0},
    }


def create_dummy_run() -> Dict[str, Dict[str, float]]:
    return {
        "1": {"doc1": 1.2, "doc2": 0.8, "doc3": 1.5},
        "2": {"doc1": 0.1, "doc2": 1.0, "doc3": 0.5},
    }


def runtest(qrels_file, run_file):
    qrels = pytrec_eval.parse_qrel(qrels_file)
    run = pytrec_eval.parse_run(run_file)
    evaluator = pytrec_eval.RelevanceEvaluator(qrels, {"map"})
    results = evaluator.evaluate(run)
    print(results)


def create_run_file(model, queries, qids, run_name):
    batch_search_output = model.search(queries, qids)
    run = []
    for qid, search_results in batch_search_output.items():
        for result in search_results:
            row_str = f"{qid} 0 {result.docno} {result.rank} {result.score} {run_name}"
            run.append(row_str)
    with open(f"outputs/{run_name}.run", "w") as f:
        for l in run:
            f.write(l + "\n")


def create_run_file2(queries, qids, run_name):
    searcher = LuceneSearcher("pyserini/indexes/full_index/")
    BM25 = searcher.set_bm25(0.9, 0.4)
    batch_search_output = BM25.search(queries, qids)


def evaluate_run(
    run: Dict[str, Dict[str, float]],
    qrels: Dict[str, Dict[str, int]],
    metric: set = {"map", "ndcg"},
) -> Dict[str, float]:
    evaluator = pytrec_eval.RelevanceEvaluator(qrels, metric)
    results = evaluator.evaluate(run)

    measures = {
        measure: np.mean(
            [query_measures.get(measure, 0) for query_measures in results.values()]
        )
        for measure in metric
    }
    return measures

def main():
    dummy_qrels = create_dummy_qrels()
    dummy_run = create_dummy_run()

    print("Dummy Qrels:")
    for qid, qrels_docs in dummy_qrels.items():
        print(f"{qid}: {qrels_docs}")

    print("\nDummy Run:")
    for qid, run_docs in dummy_run.items():
        print(f"{qid}: {run_docs}")

    evaluation_results = evaluate_run(dummy_run, dummy_qrels)
    print("\nEvaluation Results:")
    for metric, value in evaluation_results.items():
        print(f"{metric}: {value}")


if __name__ == "__main__":
    main()





In [ ]:
def find_word(text, word):
    if word in text:
        return True
    else:
        return False

def query_exists_in_text(text, query):
    words_in_text = text.lower().split()
    for word in query:
        if word.lower() in words_in_text:
            return True
    return False

# Example usage
text = "This is a sample text"
query = ["sample", "word", "example"]
exists = query_exists_in_text(text, query)
print(exists)  # Output: True


In [ ]:
def read_qrels(file_path):
    qrels = {}
    with open(file_path, "r") as f:
        for line in f:
            qid, q0, docid, rel = line.strip().split()
            if qid not in qrels:
                qrels[qid] = {}
            qrels[qid][docid] = int(rel)
    return qrels

def read_run(file_path):
    run = {}
    with open(file_path, "r") as f:
        for line in f:
            qid, _, docid, score = line.strip().split()
            if qid not in run:
                run[qid] = {}
            run[qid][docid] = float(score)
    return run


In [ ]:
import pandas as pd

qrelz =read_qrels(qrels_file)

pd.read_csv(qrels_file, sep=" ", header=None, names=["qid", "q0", "docid", "rel"],index_col=False).sort_values(by=["qid"]).head()
topics = topics.sort_values(by=["qid"]).head()
qrels = qrels.sort_values(by=["qid"]).head()


In [ ]:
pd.read_csv(queries_file, sep=" ", header=None, names=["qid", "query"],index_col=False).sort_values(by=["qid"]).head()

In [ ]:
from pyserini.index import IndexReader as index_reader

In [ ]:
def query_exists_in_text(text, query):
    words_in_text = text.lower().split()
    for word in query:
        if word.lower() in words_in_text:
            return True
    return False

# Example usage
text = "This is a sample text"
query = ["sample", "word", "example"]
exists = query_exists_in_text(text, query)
print(exists)  # Output: True


In [44]:
for qid in qrels['qid']:
    id = index_reader.convert_internal_docid_to_collection_docid(qid)
    query = index_reader.doc_contents(id)
    query = topics.loc[topics['qid'] == qid, 'query'].values[0]
    docs = index_reader.doc(id).contents( )
    print(docs[:10])
    print(query)
    print(query_exists_in_text(docs, query))

<TEXT>
htt
1 barrel equal to how much litre
True
<TEXT>
htt
1 gallon to litre
True
<TEXT>
htt
118 area code location
True
<TEXT>
htt
21th amendment definition us history
True
<TEXT>
htt
49 out of 70 percentage
True


In [45]:
searcher = LuceneSearcher("pyserini/indexes/full_index/") 
BM25 = searcher.set_bm25(0.9, 0.4)
searcher.search(query,

AttributeError: 'NoneType' object has no attribute 'search'